In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from func import *

# Read data

In [ ]:
# Read graph
edgelist = np.loadtxt("../data/portland-sub/portlandsub_edgelist.txt", dtype=np.int)

G = nx.Graph()
G.add_edges_from(edgelist)

In [ ]:
n = G.number_of_nodes()

# Create list of lists representation
ll = [[] for _ in range(n)]
for i in range(1,n+1):
    for j in G.neighbors(i):
            ll[i-1].append(int(j))

In [ ]:
# Read pre-computed edge betweenness measures
shortest_path_betweenness = read_betweenness("../data/portland-sub/portlandsub_shortest_path_betweenness.txt", edgelist)
current_flow_betweenness = read_betweenness("../data/portland-sub/portlandsub_current_flow_betweenness.txt", edgelist)
local50_betweenness = read_betweenness("../data/portland-sub/portlandsub_local50_betweenness.txt", edgelist)
local10_betweenness = read_betweenness("../data/portland-sub/portlandsub_local10_betweenness.txt", edgelist)
local02_betweenness = read_betweenness("../data/portland-sub/portlandsub_local02_betweenness.txt", edgelist)

# Read pre-computed node eigenvector centrality
values = np.loadtxt("../data/portland-sub/portlandsub_eigenvector_centrality.txt", dtype=np.float64)
eigenvector_centrality = dict((v,val) for v,val in zip(range(1,n+1),values))

# Intervention and simulation results

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

Main.include("../agentSEIR.jl")

## Scenario 1a: 85% final size, random initialization

In [ ]:
beta = 0.036

# verify that beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=200, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

##### Sample epidemic curve: random initialization, 25% intervention, no delay

In [ ]:
reduced_weight = .1
perc = .25
m = G.number_of_edges()

In [ ]:
lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
t_end = 100

init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)

_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)  

In [ ]:
t_int = np.linspace(0, t_end, num=t_end+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label='NI', linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_rw+sum_I_rw)/n, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4, alpha=1)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((0,.1,.2,.3),size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases', fontsize=22)
plt.savefig("portlandsub_curves_randinit_85.tiff", bbox_inches='tight', format='tiff', dpi=400)

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = .1
target_perc = [.05,.1,.15,.2,.25,.3,.35,.4,.45,.5]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 500

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1,.2,.3),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_randinit_85.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.6,.7,.8),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_randinit_85.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

## Scenario 1b: 85% final size, cluster initialization

In [ ]:
init_infectious = np.loadtxt("../data/portland-sub/portlandsub_initial_cluster.txt", dtype=int)

In [ ]:
beta = 0.036

# verify that beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=100, beta=beta, rand_init=False, init_infectious_nodes=init_infectious)
print(f'Total infection: {sum_R[-1]/n:.4f}')

##### Sample epidemic curve: cluster initialization, 25% intervention, no delay

In [ ]:
reduced_weight = .1
perc = .25
m = G.number_of_edges()

In [ ]:
lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
t_end = 100

_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_infectious)
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_infectious)  

In [ ]:
t_int = np.linspace(0, t_end, num=t_end+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label='NI', linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_rw+sum_I_rw)/n, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4, alpha=1)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((0,.1,.2,.3),size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases', fontsize=22)
plt.savefig("portlandsub_curves_cluster_85.tiff", bbox_inches='tight', format='tiff', dpi=400)

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = .1
target_perc = [.05,.1,.15,.2,.25,.3,.35,.4,.45,.5]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 700

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")

        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1,.2,.3),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_cluster_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.6,.7,.8),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_cluster_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Scenario 1c: 85% final size, random initialization + delay

In [ ]:
beta = 0.036

# verify beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=200, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

##### Sample epidemic curve: random initialization, 25% intervention + delay

In [ ]:
reduced_weight = .1
perc = .25
m = G.number_of_edges()

In [ ]:
lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
t_end = 80
t_delay = 5
t_con = t_end - t_delay

init_state, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_delay, beta=beta)
_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_con, beta=beta, rand_init=False, init_state=init_state)
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_state=init_state)
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_state=init_state)
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_state=init_state)
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_state=init_state)
_, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_state=init_state)
_, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_state=init_state)
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_state=init_state)
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_state=init_state) 

In [ ]:
t_ini = np.linspace(0, t_delay, num=t_delay+1)
t_int = np.linspace(t_delay, t_end, num=t_con+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_ini, (sum_E+sum_I)/n, linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label='NI', linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_rw+sum_I_rw)/n, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4, alpha=1)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((0,.1,.2,.3),size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases', fontsize=22)
plt.savefig("portlandsub_curves_randinit_85_delay5.png", bbox_inches='tight', format='png', dpi=400)

### Simulation for interventions that start on day 5

In [ ]:
reduced_weight = .1
target_perc = [.05,.1,.15,.2,.25,.3,.35,.4,.45,.5]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 300
t_delay = 5
t_con = t_end - t_delay

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        init_state, _, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_delay, beta=beta)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_con, beta=beta, rand_init=False, init_state=init_state)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_state=init_state)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_state=init_state)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_state=init_state)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_state=init_state)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_state=init_state)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_state=init_state)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_state=init_state)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_state=init_state)
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1, .2, .3), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_randinit_85_delay5.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.6, .7, .8), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_randinit_85_delay5.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

## Scenario 1d: 85% final size, random initialization + 99% final size

In [ ]:
beta = 0.036

# verify that beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=200, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = .01
target_perc = [.05,.1,.15,.2,.25,.3,.35,.4,.45,.5]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 600
m = G.number_of_edges()

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1, .2, .3), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_randinit_85_99.png", bbox_inches='tight', format='png', dpi=400)
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.4, .5, .6, .7, .8),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_randinit_85_99.png", bbox_inches='tight', format='png', dpi=400)
plt.show()

## Scenario 2: $\beta$ is set to give 70% final size

In [ ]:
beta = 0.013

# verify that beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=400, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

##### Sample epidemic curve: random initialization, 25% intervention

In [ ]:
reduced_weight = .1
perc = .25
m = G.number_of_edges()

In [ ]:
lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
t_end = 300

init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)

_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)  

In [ ]:
t_int = np.linspace(0, t_end, num=t_end+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label='NI', linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_rw+sum_I_rw)/n, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4, alpha=1)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((0, .1), size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases', fontsize=22)
plt.savefig("portlandsub_curves_randinit_70.tiff", bbox_inches='tight', format='tiff', dpi=400)

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = .1
target_perc = [.05,.1,.15,.2,.25,.3,.35,.4,.45,.5]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 750

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        # the following lines run simulation from cluster initialization
        """
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=0.055, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_ui, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_hd, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_eg, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_sp, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_rw, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_lf10, init_infectious_nodes=init_infectious, rand_init=False)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_lf02, init_infectious_nodes=init_infectious, rand_init=False)
        """
        
        # the following lines run simulation from random initialization
       
        init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1,.2),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_randinit_70.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_randinit_70.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

## Scenario 3: 55% final size, random initialization

In [ ]:
beta = 0.0085

# verify that beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=400, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

##### Sample epidemic curve: random initialization, 25% intervention

In [ ]:
reduced_weight = .1
perc = .25
m = G.number_of_edges()

In [ ]:
lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
t_end = 500

init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)

_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)  

In [ ]:
t_int = np.linspace(0, t_end, num=t_end+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label='NI', linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_rw+sum_I_rw)/n, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4, alpha=1)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((0, .1), size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases', fontsize=22)
plt.savefig("portlandsub_curves_randinit_55.tiff", bbox_inches='tight', format='tiff', dpi=400)

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = .1
target_perc = [.05,.1,.15,.2,.25,.3,.35,.4,.45,.5]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 1000

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.0, .1),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_randinit_55.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_randinit_55.tiff", bbox_inches='tight', format='tiff', dpi=400)
plt.show()

# Node immunization

In [ ]:
beta = 0.036

# verify that beta is chosen appropriately
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=200, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

In [ ]:
node_sp = edge_to_node_betweenness(shortest_path_betweenness, G.nodes())
node_rw = edge_to_node_betweenness(current_flow_betweenness, G.nodes())
node_lf50 = edge_to_node_betweenness(local50_betweenness, G.nodes())
node_lf10 = edge_to_node_betweenness(local10_betweenness, G.nodes())
node_lf02 = edge_to_node_betweenness(local02_betweenness, G.nodes())

In [ ]:
reduced_weight = 0.0
perc = .1

In [ ]:
lw_ui = create_list_of_weights_target_random_nodes(ll, G, perc, weight=reduced_weight)
lw_hd = create_list_of_weights_degree_dist_target_nodes(ll, G, perc, weight=reduced_weight)
lw_eg = create_list_of_weights_node_centrality_target_nodes(ll, eigenvector_centrality, perc, weight=reduced_weight)
lw_sp = create_list_of_weights_node_centrality_target_nodes(ll, node_sp, perc, weight=reduced_weight)
lw_rw = create_list_of_weights_node_centrality_target_nodes(ll, node_rw, perc, weight=reduced_weight)
lw_lf10 = create_list_of_weights_node_centrality_target_nodes(ll, node_lf10, perc, weight=reduced_weight)
lw_lf02 = create_list_of_weights_node_centrality_target_nodes(ll, node_lf02, perc, weight=reduced_weight)

In [ ]:
t_end = 150

init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)

_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=0.055, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)  
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=0.055, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)  

In [ ]:
t_int = np.linspace(0, t_end, num=t_end+1)

plt.figure(figsize=(9.3,5.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label='NI', linestyle='-', color='dimgray', linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_rw+sum_I_rw)/n, label='CF', linestyle=(0,(1,1)), color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4, alpha=1)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc='upper center', handlelength=2)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases', fontsize=22)
plt.savefig("portlandsub_curves_randinit_85_immu.png", bbox_inches='tight', format='png', dpi=400)

In [ ]:
reduced_weight = 0.0
target_perc = [.05,.1,.15,.2,.25,.3]
num_scenarios = len(target_perc)
num_trials = 50
t_end = 300

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist_target_nodes(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality_target_nodes(ll, eigenvector_centrality, perc, weight=reduced_weight)
    lw_sp = create_list_of_weights_node_centrality_target_nodes(ll, node_sp, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights_node_centrality_target_nodes(ll, node_rw, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights_node_centrality_target_nodes(ll, node_lf50, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights_node_centrality_target_nodes(ll, node_lf10, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights_node_centrality_target_nodes(ll, node_lf02, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/n, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/n, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/n, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/n, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/n, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/n, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/n, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/n, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/n, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((0,.1,.2,.3), size=18)
plt.xlabel('Percentage of Targeted Nodes', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("portlandsub_epipeak_randinit_85_immu.png", bbox_inches='tight', format='png', dpi=400)
plt.show()

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Nodes', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("portlandsub_episize_randinit_85_immu.png", bbox_inches='tight', format='png', dpi=400)
plt.show()